In [1]:
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from xgboost import XGBRegressor
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
import matplotlib.pyplot as plt
import os
import time
os.chdir("data")
from matplotlib import style
style.use("ggplot")


In [2]:

#getting some data
train=pd.read_csv("train.csv")
testdata=pd.read_csv("test.csv")
train_label=pd.read_csv("train_label.csv")

In [3]:
print(train.columns)
print(train.shape)
print(train_label.columns)
print(train_label.shape)

Index(['city', 'year', 'weekofyear', 'week_start_date', 'ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2',
       'reanalysis_relative_humidity_percent', 'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm'],
      dtype='object')
(1456, 24)
Index(['city', 'year', 'weekofyear', 'total_cases'], dtype='object')
(1456, 4)


In [4]:

y=train_label.total_cases
prediction_col=['city', 'year', 'weekofyear','ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2','reanalysis_relative_humidity_percent', 
        'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']
col_to_be_regularized = ['year', 'weekofyear','ndvi_ne', 'ndvi_nw',
       'ndvi_se', 'ndvi_sw', 'precipitation_amt_mm', 'reanalysis_air_temp_k',
       'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
       'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
       'reanalysis_precip_amt_kg_per_m2','reanalysis_relative_humidity_percent', 
        'reanalysis_sat_precip_amt_mm',
       'reanalysis_specific_humidity_g_per_kg', 'reanalysis_tdtr_k',
       'station_avg_temp_c', 'station_diur_temp_rng_c', 'station_max_temp_c',
       'station_min_temp_c', 'station_precip_mm']

for col in col_to_be_regularized:
    train[col] = train[col]/max(train[col])
    #plt.scatter(train[col],y)
    #plt.hist(train[col]  )
    plt.xlabel(col)
    plt.ylabel("total cases")
    #plt.show()
    #time.sleep(10)
    plt.close()
X=train[prediction_col]

In [5]:
X.describe()

,year,weekofyear,ndvi_ne,ndvi_nw,ndvi_se,ndvi_sw,precipitation_amt_mm,reanalysis_air_temp_k,reanalysis_avg_temp_k,reanalysis_dew_point_temp_k,...,reanalysis_precip_amt_kg_per_m2,reanalysis_relative_humidity_percent,reanalysis_sat_precip_amt_mm,reanalysis_specific_humidity_g_per_kg,reanalysis_tdtr_k,station_avg_temp_c,station_diur_temp_rng_c,station_max_temp_c,station_min_temp_c,station_precip_mm
count,1456.000000,1456.000000,1262.000000,1404.000000,1434.000000,1434.000000,1443.000000,1446.000000,1446.000000,1446.000000,...,1446.000000,1446.000000,1443.000000,1446.000000,1446.000000,1413.000000,1413.000000,1436.000000,1442.000000,1434.000000
mean,0.995538,0.500065,0.279909,0.287290,0.378558,0.370512,0.117154,0.988424,0.987776,0.989266,...,0.070380,0.833201,0.117154,0.818439,0.305938,0.882655,0.510084,0.769015,0.863365,0.072384
std,0.002691,0.283386,0.276442,0.264066,0.137206,0.153663,0.111919,0.004508,0.004165,0.005119,...,0.076134,0.072547,0.111919,0.075385,0.221258,0.041959,0.134719,0.046429,0.061487,0.087346
min,0.990050,0.018868,-0.799143,-1.003678,-0.028856,-0.116218,0.000000,0.974969,0.973473,0.970490,...,0.000000,0.586017,0.000000,0.572576,0.084670,0.694805,0.286618,0.632701,0.574219,0.000000
25%,0.993532,0.259434,0.088422,0.108305,0.288098,0.264110,0.025090,0.984973,0.984579,0.985488,...,0.022883,0.782650,0.025090,0.760316,0.145276,0.853896,0.412297,0.736967,0.824219,0.016013
50%,0.996020,0.500000,0.253398,0.267212,0.364192,0.346967,0.098157,0.988241,0.987986,0.990587,...,0.047756,0.814334,0.098157,0.835090,0.178253,0.890074,0.462025,0.777251,0.867187,0.043898
75%,0.997512,0.740566,0.488797,0.476643,0.462269,0.452334,0.179813,0.992169,0.991016,0.993332,...,0.091499,0.875752,0.179813,0.878639,0.475713,0.914193,0.605485,0.803318,0.910156,0.099209
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
my_imputer=Imputer()
X=pd.get_dummies(X)
X=my_imputer.fit_transform(X)
model=XGBRegressor()
X=pd.DataFrame(X)
print(X.describe())
trainX,val_X,trainy,val_y=train_test_split(X,y,random_state=0)


                0            1            2            3            4   \
count  1456.000000  1456.000000  1456.000000  1456.000000  1456.000000   
mean      0.995538     0.500065     0.279909     0.287290     0.378558   
std       0.002691     0.283386     0.257353     0.259304     0.136164   
min       0.990050     0.018868    -0.799143    -1.003678    -0.028856   
25%       0.993532     0.259434     0.109421     0.113036     0.289097   
50%       0.996020     0.500000     0.279909     0.281819     0.365953   
75%       0.997512     0.740566     0.451044     0.467235     0.458880   
max       1.000000     1.000000     1.000000     1.000000     1.000000   

                5            6            7            8            9   \
count  1456.000000  1456.000000  1456.000000  1456.000000  1456.000000   
mean      0.370512     0.117154     0.988424     0.987776     0.989266   
std       0.152497     0.111418     0.004493     0.004151     0.005102   
min      -0.116218     0.000000     0

C:\Users\Rohit\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [9]:
model.fit(X,y)
#model.fit(trainX,trainy)
del trainX,trainy,X,y
test=testdata[prediction_col]
for col in col_to_be_regularized:
    test[col] = test[col]/max(test[col])
test=pd.get_dummies(test)
test=my_imputer.fit_transform(test)
test=pd.DataFrame(test)

predicted=model.predict(test)


C:\Users\Rohit\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [10]:
#predicted=np.absolute(np.array(predicted).round())
predicted = predicted.astype(int)
print(predicted[0])
#print(val_y)
#print(mean_absolute_error(val_y,predicted))
my_submission = pd.DataFrame({'city': testdata.city, 'year': testdata.year,"weekofyear":testdata.weekofyear, 'total_cases':predicted})
my_submission.to_csv('submission.csv', index=False)


4
